In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import gspread
from oauth2client.client import GoogleCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
from googleapiclient import discovery

# Selenium

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


# Librerias

In [ ]:
#librerias
import requests #Para obtener links de internet
import re #Para utilizar Regex
import pandas as pd #Para generar Dataframes
from datetime import date
import time
import numpy as np

# Funciones

In [ ]:
def checkInternetRequests(url='https://www.google.com/', timeout=30):
    try:
        r = requests.head(url, timeout=timeout)
        return False
    except requests.ConnectionError as ex:
        print('---No hay internet---')
        return True

In [ ]:
def htmls(link):
    try:
        driver.get(link)
        time.sleep(1)
        html = driver.page_source
    except: pass
    return html

In [ ]:
def links(html):
    products = re.findall('a href="/products/(\S+)"', html)
    products = ['https://www.solotodo.cl/products/'+item for item in products]
    products = list(set(products))
    return products

In [ ]:
def to_int(df, *columnas):
        noc = [',0','$', '.','"']
        for index, row in df.iterrows():
          for column in columnas:
            try:  
                  p = row[column]
                  for c in noc:
                      try: p = p.replace(c, '')
                      except: pass
                  try:
                    df.loc[index, column] = int(p)
                  except:
                    df.drop([index], axis=0, inplace=True)
            except: continue
        return df

# Scraper Televisores


In [ ]:
def TV_take_data(links):
    data = []
    for link in links:
        no_internet = checkInternetRequests()
        while no_internet:
            no_internet = checkInternetRequests()
        driver.get(link)
        time.sleep(1)
        
        #Obtenemos el nombre del producto
        tv_name = driver.find_elements_by_css_selector("h1.mb-0")        
        tv_name = tv_name[0].text
        tv_info = [tv_name.partition(' ')[0], tv_name.partition(' ')[2]]
        
        
        tv_caract1 = driver.find_elements_by_css_selector("div#product-detail-specs div.row div dl dd")
        tv_caract2 = driver.find_elements_by_css_selector("div#product-detail-specs div.row div dl li")
        try:
            caract = [row.text for row in tv_caract1]
            caract2 = [row.text for row in tv_caract2]
            caract.append(caract2)
        except:
            caract = [np.nan]*11
      
        table = driver.find_elements_by_css_selector(" div.content-card > table.table > tbody td")
        prices = []
        for row in table:
            if row.text!='': prices.append(row.text)
        precios = [prices[i:i + 3] for i in range(0, len(prices), 3)]
        for precio in precios:
            data.append(tv_info+precio+caract)
    df = pd.DataFrame(data, columns=['Marca', 'Modelo', 'Tienda', 'Precio_Tarjeta', 'Precio_Normal', 'LED', 'Pulgadas', 'Resolucion',
                                     'SmartTV','Tasa_Refresco','Puertos USB','Tipo Panel','Parlantes','Puertos_video'])
    return df

In [ ]:
def TV_scraper(url, limit):
    print('Iniciando scraper')
    print('Obteniendo información de la página:')
    today_date = date.today()
    today_date = today_date.strftime("%Y-%m-%d")
    urls = [url + str(i)+'&' for i in range(1,limit+1)]
    DF = pd.DataFrame(columns = ['Marca', 'Modelo', 'Tienda', 'Precio_Tarjeta', 'Precio_Normal', 'LED', 'Pulgadas', 'Resolucion',
                                 'SmartTV','Tasa_Refresco','Puertos USB','Tipo Panel','Parlantes','Puertos_video'])
    i=1
    for link in urls:
        print('     N°'+str(i))
        html = htmls(link)
        products_links = links(html)
        products = TV_take_data(products_links)
        DF = DF.append(products, ignore_index=True)
        i+=1
    print('Añadiendo fechas')
    DF['Fecha'] = today_date
    
    #DF = to_int(DF, 'Precio_Tarjeta', 'Precio_Normal', 'Pulgadas')
    return DF

In [ ]:
df = TV_scraper('https://www.solotodo.cl/televisions?ordering=offer_price_usd&page=', 30)
df.info()
df.head()

Iniciando scraper
Obteniendo información de la página:
     N°1


IndexError: ignored

In [ ]:
df = to_int(df, 'Precio_Tarjeta', 'Precio_Normal', 'Pulgadas')

In [ ]:
df = df.replace(np.nan,'')

In [ ]:
df['SmartTV'] = df['SmartTV'].replace('Sí', True)
df['SmartTV'] = df['SmartTV'].replace('No', False)
    
df['Resolucion'] = df['Resolucion'].replace('HD (1366x768)', 'HD')
df['Resolucion'] = df['Resolucion'].replace('Full HD (1920x1080)', 'Full HD')
df['Resolucion'] = df['Resolucion'].replace('4k Ultra HD (3840x2160)', '4K')
df['Resolucion'] = df['Resolucion'].replace('HD+ (1600x900)', 'HD+')
df['Resolucion'] = df['Resolucion'].replace('8K Full Ultra HD (7680x4320)', '8K')

In [ ]:
df2 = df[~df.Modelo.str.contains(" \+")]

In [ ]:
df2.info()

In [ ]:
today_date = date.today()
today_date = today_date.strftime("%Y-%m-%d")
df2.to_csv('drive/MyDrive/IPC Bienes/TV/TV--'+today_date+'.csv',index=False)


# Scraper Computadores


In [ ]:
def PC_take_data(links):
    data = []
    for link in links:
        no_internet = checkInternetRequests()
        while no_internet:
            no_internet = checkInternetRequests()
        driver.get(link)
        time.sleep(1)
        
        #Obtenemos el nombre del producto
        tv_name = driver.find_elements_by_css_selector("h1.mb-0")        
        tv_name = tv_name[0].text
        tv_info = [tv_name.partition(' ')[0], tv_name.partition(' ')[2]]
        
        
        pc_caract1 = driver.find_elements_by_css_selector("div#product-detail-specs div.row div dl dd")
        #pc_caract2 = driver.find_elements_by_css_selector("div#product-detail-specs div.row div dl li")
        try:

            caract = [row.text for row in pc_caract1]
            #caract2 = [row.text for row in pc_caract2]
            #caract.append(caract2)
        except:
            caract = [np.nan]*11
      
        table = driver.find_elements_by_css_selector(" div.content-card > table.table > tbody td")
        prices = []
        for row in table:
            if row.text!='': prices.append(row.text)
        precios = [prices[i:i + 3] for i in range(0, len(prices), 3)]
        for precio in precios:
            data.append(tv_info+precio+caract)

    df = pd.DataFrame(data, columns=['Marca', 'Modelo', 'Tienda','Precio Tarjeta', 'Precio Normal','Procesador', 'RAM', 'Pantalla', 'Batería', 'Almacenamiento', 'Tarjeta video', 'Puertos', 'Peso','Dimensiones', 'Sistema Operativo', 'Unidad óptica', 'WiFi', 'Adaptador de energía', 'Lector tarjetas', 'Lector huellas'])
    return df

In [ ]:
def PC_scraper(url, limit):
    print('Iniciando scraper')
    print('Obteniendo información de la página:')
    today_date = date.today()
    today_date = today_date.strftime("%Y-%m-%d")
    urls = [url + str(i)+'&' for i in range(1,limit+1)]
    DF = pd.DataFrame(columns=['Marca', 'Modelo', 'Tienda','Precio Tarjeta', 'Precio Normal','Procesador', 'RAM', 'Pantalla', 'Batería', 'Almacenamiento', 'Tarjeta video', 'Puertos', 'Peso','Dimensiones', 'Sistema Operativo', 'Unidad óptica', 'WiFi', 'Adaptador de energía', 'Lector tarjetas', 'Lector huellas'])
    i=1
    for link in urls:
        print('     N°'+str(i))
        html = htmls(link)
        products_links = links(html)
        products = PC_take_data(products_links)
        DF = DF.append(products, ignore_index=True)
        i+=1
    print('Añadiendo fechas')
    DF['Fecha'] = today_date
    
    return DF

In [ ]:
df = PC_scraper('https://www.solotodo.cl/notebooks?ordering=offer_price_usd&page=', 75)
df.info()

In [ ]:
df.head()

In [ ]:
df = df.replace(r'\n',' ', regex=True) 

In [ ]:
df = df.replace(np.nan,'') 

In [ ]:
df = to_int(df, 'Precio Tarjeta', 'Precio Normal')

In [ ]:
df['Unidad óptica'] = df['Unidad óptica'].replace('Sí', True)
df['Unidad óptica'] = df['Unidad óptica'].replace('No', False)

df['Lector huellas'] = df['Lector huellas'].replace('Sí', True)
df['Lector huellas'] = df['Lector huellas'].replace('No', False)

In [ ]:
df2 = df[~df.Modelo.str.contains(" \+")]

In [ ]:
today_date = date.today()
today_date = today_date.strftime("%Y-%m-%d")
df2.to_csv('drive/MyDrive/IPC Bienes/Computador/PC--'+today_date+'.csv',index=False)

# Scraper Lavadoras

In [ ]:
def LS_take_data(links):
    data = []
    for link in links:
        no_internet = checkInternetRequests()
        while no_internet:
            no_internet = checkInternetRequests()
        driver.get(link)
        time.sleep(1)
        
        #Obtenemos el nombre del producto
        x_name = driver.find_elements_by_css_selector("h1.mb-0")        
        x_name = x_name[0].text
        x_info = [x_name.partition(' ')[0], x_name.partition(' ')[2]]
        
        
        x_caract1 = driver.find_elements_by_css_selector("div#product-detail-specs div.row div dl dd")
        try:
            caract = [row.text for row in x_caract1]
        except:
            caract = [np.nan]*10
      
        table = driver.find_elements_by_css_selector(" div.content-card > table.table > tbody td")
        prices = []
        for row in table:
            if row.text!='': prices.append(row.text)
        precios = [prices[i:i + 3] for i in range(0, len(prices), 3)]
        for precio in precios:
            data.append(x_info+precio+caract)
    df = pd.DataFrame(data, columns=['Marca', 'Modelo', 'Tienda', 'Precio_Tarjeta', 'Precio_Normal','Tipo','Potencia','Capacidad Lavado', 'Capacidad Secado','Tamaño', 'Peso', 'Carga', 'Color', 'Controles', 'Material Tambor'])
    return df

In [ ]:
def LS_scraper(url, limit):
    print('Iniciando scraper')
    print('Obteniendo información de la página:')
    today_date = date.today()
    today_date = today_date.strftime("%Y-%m-%d")
    urls = [url + str(i)+'&' for i in range(1,limit+1)]
    DF = pd.DataFrame(columns = ['Marca', 'Modelo', 'Tienda', 'Precio_Tarjeta', 'Precio_Normal','Tipo','Potencia','Capacidad Lavado', 'Capacidad Secado','Tamaño', 'Peso', 'Carga', 'Color', 'Controles', 'Material Tambor'])
    i=1
    for link in urls:
        print('     N°'+str(i))
        html = htmls(link)
        products_links = links(html)
        products = LS_take_data(products_links)
        DF = DF.append(products, ignore_index=True)
        i+=1
    print('Añadiendo fechas')
    DF['Fecha'] = today_date
    
    #DF = to_int(DF, 'Precio_Tarjeta', 'Precio_Normal', 'Pulgadas')
    return DF

In [ ]:
df = LS_scraper('https://www.solotodo.cl/washing_machines?ordering=offer_price_usd&page=', 20)
df.info()
df.head()

Iniciando scraper
Obteniendo información de la página:
     N°1
     N°2
     N°3


IndexError: ignored

In [ ]:
df = to_int(df, 'Precio_Tarjeta', 'Precio_Normal')

In [ ]:
df = df.replace(np.nan,'')

In [ ]:
df2 = df[~df.Modelo.str.contains(" \+")]

In [ ]:
today_date = date.today()
today_date = today_date.strftime("%Y-%m-%d")
df2.to_csv('drive/MyDrive/IPC Bienes/Lavadora - Secadora/LS--'+today_date+'.csv',index=False)

# Scraper Refrigeradores

In [ ]:
def RF_take_data(links):
    data = []
    for link in links:
        no_internet = checkInternetRequests()
        while no_internet:
            no_internet = checkInternetRequests()
        driver.get(link)
        time.sleep(1)
        
        #Obtenemos el nombre del producto
        x_name = driver.find_elements_by_css_selector("h1.mb-0")        
        x_name = x_name[0].text
        x_info = [x_name.partition(' ')[0], x_name.partition(' ')[2]]
        
        
        x_caract1 = driver.find_elements_by_css_selector("div#product-detail-specs div.row div dl dd")
        try:
            caract = [row.text for row in x_caract1]
        except:
            caract = [np.nan]*10
      
        table = driver.find_elements_by_css_selector(" div.content-card > table.table > tbody td")
        prices = []
        for row in table:
            if row.text!='': prices.append(row.text)
        precios = [prices[i:i + 3] for i in range(0, len(prices), 3)]
        for precio in precios:
            data.append(x_info+precio+caract)
    df = pd.DataFrame(data, columns=['Marca', 'Modelo', 'Tienda', 'Precio_Tarjeta', 'Precio_Normal','Tipo', 'Frosting', 'Eficiencia', 'Consumo', 'Capacidad Refrigerador', 'Capacidad Freezer', 'Tamaño', 'Peso', 'Dispensador de agua', 'Dispensador de hielo'])
    return df

In [ ]:
def RF_scraper(url, limit):
    print('Iniciando scraper')
    print('Obteniendo información de la página:')
    today_date = date.today()
    today_date = today_date.strftime("%Y-%m-%d")
    urls = [url + str(i)+'&' for i in range(1,limit+1)]
    DF = pd.DataFrame(columns = ['Marca', 'Modelo', 'Tienda', 'Precio_Tarjeta', 'Precio_Normal','Tipo', 'Frosting', 'Eficiencia', 'Consumo', 'Capacidad Refrigerador', 'Capacidad Freezer', 'Tamaño', 'Peso', 'Dispensador de agua', 'Dispensador de hielo'])
    i=1
    for link in urls:
        print('     N°'+str(i))
        html = htmls(link)
        products_links = links(html)
        products = RF_take_data(products_links)
        DF = DF.append(products, ignore_index=True)
        i+=1
    print('Añadiendo fechas')
    DF['Fecha'] = today_date
    
    #DF = to_int(DF, 'Precio_Tarjeta', 'Precio_Normal', 'Pulgadas')
    return DF

In [ ]:
df = RF_scraper('https://www.solotodo.cl/refrigerators?ordering=offer_price_usd&page=', 24)
df.info()
df.head()

In [ ]:
df = to_int(df, 'Precio_Tarjeta', 'Precio_Normal')
df = df.replace(np.nan,'')

df['Dispensador de hielo'] = df['Dispensador de hielo'].replace('Sí', True)
df['Dispensador de hielo'] = df['Dispensador de hielo'].replace('No', False)
df['Dispensador de agua'] = df['Dispensador de agua'].replace('Sí', True)
df['Dispensador de agua'] = df['Dispensador de agua'].replace('No', False)

df2 = df[~df.Modelo.str.contains(" \+")]

In [ ]:
today_date = date.today()
today_date = today_date.strftime("%Y-%m-%d")
df2.to_csv('drive/MyDrive/IPC Bienes/Refrigerador/RF--'+today_date+'.csv',index=False)

# Scraper Impresoras

In [ ]:
def IM_take_data(links):
    data = []
    for link in links:
        no_internet = checkInternetRequests()
        while no_internet:
            no_internet = checkInternetRequests()
        driver.get(link)
        time.sleep(1)
        
        #Obtenemos el nombre del producto
        x_name = driver.find_elements_by_css_selector("h1.mb-0")        
        x_name = x_name[0].text
        x_info = [x_name.partition(' ')[0], x_name.partition(' ')[2]]
        
        
        x_caract1 = driver.find_elements_by_css_selector("div#product-detail-specs div.row div dl dd")
        try:
            caract = [row.text for row in x_caract1]
        except:
            caract = [np.nan]*9
      
        table = driver.find_elements_by_css_selector(" div.content-card > table.table > tbody td")
        prices = []
        for row in table:
            if row.text!='': prices.append(row.text)
        precios = [prices[i:i + 3] for i in range(0, len(prices), 3)]
        for precio in precios:
            data.append(x_info+precio+caract)
    df = pd.DataFrame(data, columns=['Marca', 'Modelo', 'Tienda', 'Precio_Tarjeta', 'Precio_Normal','Tipo', 'Tecnologia', 'Color', 'Puerto', 'Velocidad ByN', 'Velocidad Color', 'Duplex', 'Red', 'Funciones'])
    return df

In [ ]:
def IM_scraper(url, limit):
    print('Iniciando scraper')
    print('Obteniendo información de la página:')
    today_date = date.today()
    today_date = today_date.strftime("%Y-%m-%d")
    urls = [url + str(i)+'&' for i in range(1,limit+1)]
    DF = pd.DataFrame(columns=['Marca', 'Modelo', 'Tienda', 'Precio_Tarjeta', 'Precio_Normal','Tipo', 'Tecnologia', 'Color', 'Puerto', 'Velocidad ByN', 'Velocidad Color', 'Duplex', 'Red', 'Funciones'])
    i=1
    for link in urls:
        print('     N°'+str(i))
        html = htmls(link)
        products_links = links(html)
        products = IM_take_data(products_links)
        DF = DF.append(products, ignore_index=True)
        i+=1

    DF = DF.replace(r'\n',' ', regex=True) 

    redes = ['Bluetooth', 'Ethernet', 'LAN', 'LAN (inalámbrica)', 'NFC', 'Wi-Fi']
    funciones = ['Copiadora', 'Escáner', 'Fax']

    for index, row in DF.iterrows():

      for red in redes:
        value = row['Red']
        if not isinstance(value, str):
          continue

        if (red in value):
          DF.loc[index, red] = True
        else:
          DF.loc[index, red] = False
      for funcion in funciones:
        value = row['Funciones']
        if not isinstance(value, str):
          continue
        if funcion in value:
          DF.loc[index, funcion] = True
        else:
          DF.loc[index, funcion] = False

    DF.drop(['Funciones', 'Red'], axis=1, inplace=True)
    print('Añadiendo fechas')
    DF['Fecha'] = today_date
    
    #DF = to_int(DF, 'Precio_Tarjeta', 'Precio_Normal', 'Pulgadas')
    return DF

In [ ]:
df = IM_scraper('https://www.solotodo.cl/printers?ordering=offer_price_usd&page=', 20)
df.info()
df.head()

In [ ]:
df['Duplex'] = df['Duplex'].replace('Sí', True)
df['Duplex'] = df['Duplex'].replace('No', False)

df2 = df[~df.Modelo.str.contains(" \+")]

In [ ]:
today_date = date.today()
today_date = today_date.strftime("%Y-%m-%d")
df2.to_csv('drive/MyDrive/IPC Bienes/Impresora/IM--'+today_date+'.csv',index=False)

# Scraper Celulares

In [ ]:
def Celular_take_data(links):
    data = []
    for link in links:
        no_internet = checkInternetRequests()
        while no_internet:
            no_internet = checkInternetRequests()
        driver.get(link)
        time.sleep(2)
        
        #Obtenemos el nombre del producto
        cell_name = driver.find_elements_by_css_selector("h1.mb-0")        
        cell_name = cell_name[0].text
        cell_info = [cell_name.partition(' ')[0], cell_name.partition(' ')[2]]
        
        
        cell_caract1 = driver.find_elements_by_css_selector("div#product-detail-specs div.row div dl dd")
        try:
            caract = [row.text for row in cell_caract1]
        except:
            caract = [np.nan]*11
      
        table = driver.find_elements_by_css_selector(" div.content-card > table.table > tbody td")
        prices = []
        for row in table:
          prices.append(row.text)

        noc = [',0','$ ', '.','"']
        for i in range(len(prices)):
          for word in noc:
            try:
              prices[i] = prices[i].replace(word,'')
            except:
              continue
          try:
            prices[i] = int(prices[i])
          except:
            continue

        dprecios = []
        x = list(range(len(prices)))
        while len(x)>=2:
          if (type(prices[x[0]]) == str) and (type(prices[x[1]]) == str):
            if (type(prices[x[2]]) == int):
              tienda = prices[x[0]]
              base = x[1]
              dprecios.append([prices[x[0]], prices[x[1]], prices[x[2]], prices[x[3]]])
              for j in range(0,3):  
                x.remove(base + j)
            elif (type(prices[x[2]]) == str):
              x.remove(x[0])
          else:
            break
        for precio in dprecios:
          data.append(cell_info + precio + caract)
    df = pd.DataFrame(data, columns=['Marca', 'Modelo' , 'Tienda', 'Contrato', 'Precio Tarjeta', 'Precio Normal', 'Pantalla', 'Memoria interna', 'RAM', 'Cámara principal', 'Cámara frontal', 'Memoria externa', 'Batería', 'Sistema Operativo', 'Procesador', 'CPU', 'GPU', 'Peso', 'Tamaño', 'Conectividad', 'Potencia carga', 'Conector audifonos', 'Wi-Fi', 'GPS', 'NFC', 'Bluetooth', 'Lector huellas'])
    return df

In [ ]:
def Celular_scraper(url, limit):
    print('Iniciando scraper')
    print('Obteniendo información de la página:')
    today_date = date.today()
    today_date = today_date.strftime("%Y-%m-%d")
    urls = [url + str(i)+'&' for i in range(1,limit+1)]
    DF = pd.DataFrame(columns = ['Marca', 'Modelo' , 'Tienda', 'Contrato', 'Precio Tarjeta', 'Precio Normal', 'Pantalla', 'Memoria interna', 'RAM', 'Cámara principal', 'Cámara frontal', 'Memoria externa', 'Batería', 'Sistema Operativo', 'Procesador', 'CPU', 'GPU', 'Peso', 'Tamaño', 'Conectividad', 'Potencia carga', 'Conector audifonos', 'Wi-Fi', 'GPS', 'NFC', 'Bluetooth', 'Lector huellas'])
    i=1
    for link in urls:
        print('     N°'+str(i))
        html = htmls(link)
        products_links = links(html)
        products = Celular_take_data(products_links)
        DF = DF.append(products, ignore_index=True)
        i+=1
    print('Añadiendo fechas')
    DF['Fecha'] = today_date
    DF = DF.replace(r'\n',' ', regex=True) 
    #DF = to_int(DF, 'Precio_Tarjeta', 'Precio_Normal', 'Pulgadas')
    return DF

In [ ]:
df = Celular_scraper('https://www.solotodo.cl/cells?ordering=offer_price_usd&page=', 50)

In [ ]:
df.head()

In [ ]:
columnas = ['Conector audifonos', 'Wi-Fi', 'GPS', 'NFC', 'Bluetooth', 'Lector huellas']
for col in columnas:
  df[col] = df[col].replace('Sí', True)
  df[col] = df[col].replace('No', False)

df2 = df[~df.Modelo.str.contains(" \+")]

In [ ]:
today_date = date.today()
today_date = today_date.strftime("%Y-%m-%d")
df2.to_csv('drive/MyDrive/IPC Bienes/Celulares/C--'+today_date+'.csv',index=False)

# Consolidado Televisores

In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1l1qdtJWAD4tZJc4p8wXokapnvglLTWG401RTe6u9KuQ')

In [ ]:
df = pd.read_csv('drive/MyDrive/IPC Bienes/TV/TV--'+today_date+'.csv')

In [ ]:
values = df.values.tolist()
worksheet.values_append('data', {'valueInputOption': 'USER_ENTERED'}, {'values': values})

# Consolidado Computadores


In [ ]:
today_date = date.today()
today_date = today_date.strftime("%Y-%m-%d")

In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1TstykekN-tpkZ4TrmLerdhqaJr3xKQuf2-l0EwuEBFM')

In [ ]:
df = pd.read_csv('drive/MyDrive/IPC Bienes/Computador/PC--'+today_date+'.csv')
df = df.replace(np.nan,'') 

In [ ]:
values = df.values.tolist()
worksheet.values_append('data', {'valueInputOption': 'USER_ENTERED'}, {'values': values})

{'spreadsheetId': '1TstykekN-tpkZ4TrmLerdhqaJr3xKQuf2-l0EwuEBFM',
 'tableRange': 'data!A1:U10214',
 'updates': {'spreadsheetId': '1TstykekN-tpkZ4TrmLerdhqaJr3xKQuf2-l0EwuEBFM',
  'updatedCells': 19278,
  'updatedColumns': 21,
  'updatedRange': 'data!A10215:U11132',
  'updatedRows': 918}}